In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import numpy as np
import tensorflow as tf
import keras

img_rows, img_cols = 299, 299

base_model = keras.applications.Xception(
    weights='imagenet',
    input_shape=(img_rows, img_cols, 3),
    include_top=False) # exclude the classification layers; keep the features layers

base_model.trainable = True

In [8]:
num_classes=20
inputs = keras.Input(shape=(img_rows, img_cols, 3))
x = tf.keras.applications.xception.preprocess_input(inputs)
x = base_model(x, training=True)
# apply the new other classification layers to the feature layers
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [11]:
from keras.src.legacy.preprocessing.image import ImageDataGenerator
input_shape = (img_rows, img_cols, 3)

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_train',target_size=(img_rows,img_cols),batch_size=20,class_mode='categorical', shuffle=False)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_val',target_size=(img_rows,img_cols),batch_size=20,class_mode='categorical', shuffle=False)

Found 200 images belonging to 20 classes.
Found 100 images belonging to 20 classes.


In [12]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.0004),
              metrics=['accuracy'])

from keras.callbacks import ModelCheckpoint
filepath = "/content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_model/model_faceID.keras"
checkpoint = ModelCheckpoint(filepath, verbose=1, mode='max')
callbacks_list = [checkpoint]

history = model.fit(train_generator,epochs=20,
                   validation_data=val_generator,
                   callbacks=callbacks_list)

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 36s/step - accuracy: 0.0000e+00 - loss: 3.0973 
Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_model/model_faceID.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 499s 46s/step - accuracy: 0.0000e+00 - loss: 3.0980 - val_accuracy: 0.0500 - val_loss: 3.0411
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 36s/step - accuracy: 0.0714 - loss: 2.9954 
Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_model/model_faceID.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 426s 42s/step - accuracy: 0.0695 - loss: 2.9954 - val_accuracy: 0.0100 - val_loss: 2.9957
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 37s/step - accuracy: 0.0423 - loss: 2.9957 
Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/Intro to Image Processing (10-11 8 2024)/gt_db_model/model_faceID.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 462s 47s/step - accuracy: 0.0430 - loss: 2.9958 - val_accuracy: 0.05

KeyboardInterrupt: 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
# Plot training accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Validate'], loc='upper left')
plt.savefig('Accuracy.png')
plt.show()

# Plot training loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Validate'], loc='upper left')
plt.savefig('Loss.png')
plt.show()